This Python 3 environment comes with many helpful analytics libraries installed
# About 
  - NutritionBOT is an interactive healthy eating expert designed to help Indian food enthusiasts and individuals to adopt and find
    alternatives suitable for their diet and lifestyle.
 - mission is to guide users through a practical, easy-to-follow process to build wholesome meal plans tailored to various dietary needs.
  ## What else can it do?
  - This chatbot, powered by generative AI and grounded with real-time web search, provides personalized Indian nutrition advice and recipe suggestions based on user preferences like spice level and allergies. It engages users in interactive conversations, offering tailored meal plans and guiding them through dietary choices in a conversational manner.
## What GenAI capabilities the bot has? 
- Agents: Although not autonomous. This is an agent that interacts with the user generates outputs.
- Long context window: It uses context window to track user input and respond based on previous messages. The conversation state is maintained in the NutriQuery state structure.
- Structured output: output is in a structured format, especially when creating recipes.
- Few-shot prompting via detailed instruction instructions in system message. 

# Necessary Libraries: 
To support the NutriBOT’s functionality,the following key libraries are imported :
mesop: for UI interaction and state management.
langchain: for integrating generative AI and tools such as Google search.
logging:  for tracking chatbot performance and user feedback (MLOps).
Other utilities:  for message handling, agent execution, and state management.

In [ ]:
# Remove conflicting packages from the Kaggle base environment.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
# Install langgraph and the packages used in this lab.
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'
!pip install -U -q "google-genai==1.9.0" 
 

In [ ]:
!pip install -qU "google-genai==1.9.0"

In [ ]:
!pip install   pinecone-client pillow graphviz requests  langchain-core

In [ ]:
import base64
import os
from google import genai
from google.genai import types
from IPython.display import display, HTML, Markdown
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from langchain_core.messages.ai import AIMessage
from langchain_core.messages import SystemMessage, HumanMessage
import requests
from pprint import pprint

# Automated retry
Set up a retry helper. This allows  to "Run all" without worrying about per-minute quota.

In [ ]:
from google.api_core import retry
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

# Set up the API Key

In [ ]:
from kaggle_secrets import UserSecretsClient
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Define Stateobject.
##### 1 Defining the Stateobject, NutriQuery, that holds the conversation history, a flag indicating if the user has exhausted his questions about Indian Nutrition. 
##### 2 order:represents the order or steps in the conversation (used for controlling the flow).
##### 3 finished: A flag to indicate if the conversation should be ended (based on user input like "exit" or "thank you").

#### Added details.
- Role: The chatbot is called DesiNutritionBot, a friendly expert on healthy eating, especially for Indian families living in the USA.
- Mission: To help users build personalized meal plans that fit their unique dietary needs.
- Capabilities: It can create meal plans, suggest food swaps, share nutritional info, and more.
- Process: It follows a step-by-step method to guide users through the conversation smoothly.
- Tone and Style: The chatbot keeps things simple, friendly, and culturally relevant — using bullet points, easy language, and visuals when possible.
- Community Focus: It understands common health challenges and offers practical lifestyle tips tailored to Indian-American families.
- Closing: It wraps up chats with a warm and friendly goodbye.
- Tool Handling: If some features or tools aren’t available, it knows how to respond politely and clearly.

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages


class NutriQuery(TypedDict):
    """State representing the customer's enquiry about Indian Nutrition in USA ."""

    # BOT conversation. 
    # Preserves the conversation history between nodes. The `add_messages` annotation  to LangGraph
    # the state is updated by  appending and not replacing them.
    messages: Annotated[list, add_messages]
  
    # The user input list
    order: list[str]

    #  value set if the user done? true or false
    finished: bool

# This defines how the chatbot is expected to behave and includes
# rules for when to call different functions, as well as rules for the conversation, such
# as tone and what is permitted for discussion.

NUTRIABOT_SYSINT = (
     "system",  # 'system' indicates the message is a system instruction. 
    "You are a DesiNutritionBot, an interactive healthy eating expert designed to help Indian families and individuals living in the USA adopt and "
    "\n\n"
    "Your mission is to guide users through a practical, easy-to-follow process to build wholesome meal plans tailored to various dietary needs  "
    "—including Indian Vegetarian, Vegan, Keto (Desi twist!), Paleo, Diabetic-friendly, and Gluten- "
    "free diets. "
     "\n\n"
    "You’ll explain, suggest, and create: "
    "\n\n"
    "- Indianized meal plans for each dietary type\n- Food lists: what to include or avoid for each lifestyle \n "
    "- Ingredient substitutions available in the U.S. (e.g., kale for thotakura, almond flour for besan)"
    "\n\n"
    "- Nutritional values for Indian-style meals\n- Step-by-step instructions for meal prep and planning\n\nStart your journey with:\n "
    
    "Here’s how you’ll proceed:\n "
    "1. Understand Dietary Lifestyles" 
      "-Research and explain the principles of different diets"  
      "-Include Indian food compatibility and tips for each"
      "\n\n"
    "2. Curate Foods and Substitutes"
      "-List pantry staples per diet" "\n"    
     "-Offer U.S.-based substitutes" "\n"    
     "- Add Telugu/Hindi/Tamil names where helpful"
      "\n\n"
    "3. Design Sample Meal Plans (Desi Edition)"
      "- Full-day plans (breakfast, lunch, dinner, snacks)"     
      "- Portion sizes and calorie counts"    
      "- Separate plans for working adults, kids, elders" 
      "- Quick weekday meals and weekend specials"
     "\n\n"
    "4. Make It Practical & Engaging" 
      "- Tips for grocery shopping in U.S. Indian/Asian stores" 
      "- Hacks for batch cooking Indian meals" 
      "- Smart eating out options (Indian + American cuisines)" 
      "- Healthy spins on favorite desi dishes (e.g., air-fried pakoras, quinoa upma)"
     "\n\n"
   "5. Use Simple and Friendly Language"  
    "- Use bullet points, clear visuals, and charts" "\n"    
    "- Include cultural and festive food tips (Ayurvedic swaps, festival food tweaks)" 
     "\n\n"
  "6.Keep the Community in Mind"
    "- Focus on common health concerns (diabetes, high cholesterol, weight)" 
    "- Offer lifestyle tweaks like mindful eating and post-meal walks" 
     "\n\n"
    "You are not just sharing nutritional advice—you’re a cultural bridge helping Indians in America stay healthy *and* stay rooted." 
     "\n\n"
       "Feel free to ask questions, clarify, or share updates with your user as they build their healthy eating guide."
       "End every session with a warm note and:" 
     "\n\n"
    "Say goodbye!"
    "\n\n"
    "If any of the tools are unavailable, you can break the fourth wall and tell"
    "the user that they have not implemented them yet and should keep reading to do so."
    ,
)


# Welcome message with which the system opens the conversation.
WELCOME_MSG = "Welcome to the Nutrition BOT. Type `q` to quit. I am a nutrition expert, here to help desis in the USA eat healthy, stay fit, and enjoy their food!?. You can start by saying *food* "

chatbot() function takes the current conversation as input.
 ###### what does it do?
 - It constructs the full message history by combining the initial NUTRIABOT_SYSINT with the existing state.
 - llm.invoke(message_history) sends the entire conversation history to the Gemini model to get the next AI response.
 - It returns a new state with the model's response as a single-element list in the messages field.

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI

# Using The Gemini 2.0 flash model #instantiates the Gemini 2.0 Flash model through the LangChain integration. 
#This is the core generative AI model being used.
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")


def chatbot(state: NutriQuery) -> NutriQuery:
    """ A simple wrapper around the model's own chat interface."""
    message_history = [NUTRIABOT_SYSINT] + state["messages"]
    return {"messages": [llm.invoke(message_history)]}


# Setting up the initial graph based on  state definition.
graph_builder = StateGraph(NutriQuery) # initializes a graph that manages the conversation

#adds the chatbot function as a node in the graph, named "chatbot".
graph_builder.add_node("chatbot", chatbot) 
# Chatbot app entrypoint.
graph_builder.add_edge(START, "chatbot")
#The code then visualizes the graph using Mermaid and displays it.
chat_graph = graph_builder.compile()

In [ ]:
# visualizing the chatbot 
from IPython.display import Image, display
Image(chat_graph.get_graph().draw_mermaid_png())

In [ ]:
from pprint import pprint

#user input message 
user_msg = "Hello"
state = chat_graph.invoke({"messages": [user_msg]})

# Uncomment the pprint lines to see all the information in state.
#pprint(state)

# final state now has 2 messages. Our HumanMessage, and an additional AIMessage.
for msg in state["messages"]:
    print(f"{type(msg).__name__}: {msg.content}")

### More Nodes
- human_node () simulates the human interaction in the conversation and retrieves, prints the last message from the model.
- verifies if user input wants to quit and sets the finished flag in the state accordingly.

In [ ]:
from langchain_core.messages.ai import AIMessage


def human_node(state: NutriQuery) -> NutriQuery:
    """Showing the last model message to the user, and receive the user's input."""
    last_msg = state["messages"][-1]
    print("Model:", last_msg.content)

    user_input = input("User: ")

    # If the user is trying to quit, notify conversation as over 
    if user_input in {"q", "quit", "exit", "goodbye", "done", "time to eat", "i am good", "i am done", "thank you","thanks"}:
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}


def chatbot_with_welcome_msg(state: NutriQuery) -> NutriQuery:
    """ Creating a wrapper around the model's own chat interface."""

    if state["messages"]:
        # If user inputs a messages, continue the conversation with the Gemini model.
        new_output = llm.invoke([NUTRIABOT_SYSINT] + state["messages"])
    else:
        # If there are no messages, start with the welcome message.
        new_output = AIMessage(content=WELCOME_MSG)

    return state | {"messages": [new_output]}

 #----Building the Interactive LangGraph----
# Start building a new graph.
graph_builder = StateGraph(NutriQuery)

# Add the chatbot and human nodes to the app graph.
graph_builder.add_node("chatbot", chatbot_with_welcome_msg)
graph_builder.add_node("human", human_node)

# Start with the chatbot again.
graph_builder.add_edge(START, "chatbot")

# The chatbot will always go to the human next.
graph_builder.add_edge("chatbot", "human");

- maybe_exit_human_node() checks the finished flag in the state. If user wants to quit, it directs the flow to END, otherwise, it goes back to the "chatbot" node.
- The graph is visualized again.

In [ ]:
from typing import Literal


def maybe_exit_human_node(state: NutriQuery) -> Literal["chatbot", "__end__"]:
    """back to the chatbot, unless the user is exiting."""
    if state.get("finished", False):
        return END
    else:
        return "chatbot"


graph_builder.add_conditional_edges("human", maybe_exit_human_node)
chat_with_human_graph = graph_builder.compile()


Image(chat_with_human_graph.get_graph().draw_mermaid_png()) 

In [ ]:
# set to default recursion limit for traversing nodes, tested for limt of 100 too. 

config = {"recursion_limit": 25}

#  unless you input `q`, `quit` or one of the other exit terms defined in `human_node` the chat will continue.
# Uncomment this line to execute the graph:
state = chat_with_human_graph.invoke({"messages": []}, config)

# Things to try:
#  - Just chat! There is no.
#  - 'q' to exit.
#pprint(state)

# Future Improvements.
- Ability to accept user preferences of food. Ex: Spice levels, allergens.
- Ability to predict recipe from image
- Ability to generate receipe from youtube video and provide subsitutes based on user preferences.
- Ability to create meal planning based on budget, season etc.

# Footnote:
- The 5 day intensive course is a great experience for anyone who have limited or no background with GenAI.
- Link to the course I took https://rsvp.withgoogle.com/events/google-generative-ai-intensive_2025q1
# Acknowledgment & Thank you 

A big shoutout to all who made this course a wonderful experience, with special mention to the moderators of Discord and Special thanks to  Anant Nawalgaria, and Paige Bailey along with .
Mat Velloso, 
Irina Sigler, 
Logan Kilpatrick,
Kieran Milan, 
Warren Barkley (Jeff Dean earlier)
André Araújo,
Patricia Florissi, Ph. D., 
Alan Li, 
Xiaoqi Ren, 
Chuck Sugnet, 
Howard Zhou,
Julia Wiesinger, 
Patrick Marlow, 
Vladimir Vuskovic,
Antonio Gulli,
Lavi Nigam, 
Julia Wiesinger, 
Vladimir Vuskovic, 
Irina Sigler, 
Ivan 🥁 Nardini, 
Nicolas Stroppa, 
Dr. Sokratis Kartakis, 
Narek Saribekyan, for the amazing whitepaper on Agents Companion! You all are the real MVPs!
And my sincere apologies to those that i've missed to mention. 